In [1]:
import numpy as np

In [2]:
def knapsack(weight, value, capacity):
    if len(weight) != len(value):
        raise ValueError('weight and value lists must have same length!')
    if capacity < 0:
        return -np.inf
    if capacity == 0:
        return 0
    if len(weight) == 0:
        return 0
     
    return max(knapsack(weight[:-1], value[:-1], capacity), 
               knapsack(weight[:-1], value[:-1], capacity-weight[-1]) + value[-1])

In [3]:
w = [1, 2, 3, 8, 7, 4]
v = [20, 5, 10, 40, 15, 25]
cap = 10

In [4]:
knapsack(w, v, cap)

60

### Ok, but what items do I want?  

While the previous method works to give us the maximum possible value of the items we can select, it doesn't provide us with a way to determine _what collection of items_ will give us that value.  

It doesn't seem so straightforward to modify that recursive method to give the collection of items that gives the maximum value.  

By writing a new method to determine the maximum value in a different fashion (by computing a table of values), we can trace back through the table to determine a collection of items that we want to take.  

**Note:**  This isn't the most succinct code that I could have written, but I wanted to illustrate the method for filling in the table and finding the collection of items that give that maximum value.  Shorter code would consist of more list comprehensions, etc.  

In [5]:
from collections import namedtuple

item = namedtuple('item', ['w','v'])

In [6]:
def knapsack(weight, value, capacity, show_table=False, show_items=True):
    if len(weight) != len(value):
        raise ValueError('weight and value lists must have same length!')
    if capacity < 0:
        return -np.inf
    if capacity == 0:
        return 0
    if len(weight) == 0:
        return 0
    
    #  Each column is a maximum capacity value we can carry.
    #  Rows correspond to adding an additional item into consideration.
    #  Fill in the first row
    row = [value[0] if weight[0] <= k else 0 
           for k in range(capacity+1)]
    M = [row]
    
    #  Now fill in the rest of the rows
    for i, w, v in zip(range(1,len(weight[1:])+1), weight[1:], value[1:]):
        row = []
        for k in range(capacity+1):
            row.append(max(M[i-1][k], 
                      (M[i-1][k-w] + v if k-w >= 0 else 0)))
        M.append(row)
    
    current_capacity = cap
    items = []
    for r, w, v in zip(range(len(weight)-1, -1, -1), weight[::-1], value[::-1]):
        if (current_capacity-w >= 0) and \
            M[r][current_capacity] == (M[r-1][current_capacity-w] + v):
            items.append(item(w,v))
            current_capacity -= w
    
    if show_table:
        print(M)

    if show_items:
        return M[-1][-1],tuple(items)
    else:
        return M[-1][-1]

In [7]:
knapsack(w, v, cap)

(60, (item(w=4, v=25), item(w=3, v=10), item(w=2, v=5), item(w=1, v=20)))